#### Dynamic Capabilities

In [0]:
# We will use widgets to make our notebook more interactive and dynamic. Now, we can process the data from different files to bronze layer
dbutils.widgets.text("file_name", "bronze")
p_file_name = dbutils.widgets.get("file_name")
print(p_file_name)

#### Data Reading

In [0]:
# to read data from ADLS gen2 in batch load
df = spark.read.parquet("abfss://source@dldatabricks.dfs.core.windows.net/customers")
# display(df)

In [0]:
# Reading data as a stream using Autloader
## schemaLocation and checkpointLocation has to be match
df = spark.readStream.format("cloudFiles")  \
            .option("cloudFiles.format", "parquet") \
            .option("cloudFiles.schemaLocation", f"abfss://bronze@dldatabricks.dfs.core.windows.net/checkpoint_{p_file_name}") \
            .load(f"abfss://source@dldatabricks.dfs.core.windows.net/{p_file_name}")


#### Data Writing

In [0]:
# Writing the stream data
## schemaLocation and checkpointLocation has to be match
### trigger(once = True) is used to read the data once and write it to the sink and if we want make it continuous we can remove the trigger option
df.writeStream.format("parquet")\
    .option("mergeSchema", "true") \
    .outputMode("append")\
    .option("checkpointLocation", f"abfss://bronze@dldatabricks.dfs.core.windows.net/checkpoint_{p_file_name}") \
    .option("path", f"abfss://bronze@dldatabricks.dfs.core.windows.net/{p_file_name}")\
    .trigger(once = True) \
    .start()